# Acquire the Raw Data

In [24]:
import pandas_datareader.data as web
import datetime
from pathlib import Path
import pandas as pd
import json

In [25]:
def acquire_training_data():
    start = datetime.datetime(2019, 7, 1)
    end = datetime.datetime(2019, 9, 30)
    df = web.DataReader("BTC-USD", 'yahoo', start, end)
    return df

In [39]:
data = acquire_training_data()


In [40]:
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-07-01,11206.439453,10089.314453,10796.930664,10583.134766,29378589324,10583.134766
2019-07-02,10912.188477,9737.884766,10588.683594,10801.677734,31015895223,10801.677734
2019-07-03,11968.078125,10818.156250,10818.156250,11961.269531,30796494294,11961.269531
2019-07-04,12006.075195,11166.569336,11972.718750,11215.437500,25920294033,11215.437500
2019-07-05,11395.661133,10874.964844,11203.102539,10978.459961,23838480210,10978.459961


In [28]:
def write_data_to_lake(df, path):
    json_data = df.to_json(orient="records")
    with open(path, 'w+') as f:
        json.dump(json_data, f, indent=4)

In [30]:
WRITE_PATH = Path().resolve().parent
WRITE_PATH = WRITE_PATH.joinpath("datalake", "2019-07-01.json")

In [31]:
write_data_to_lake(data, WRITE_PATH)

# Send Data into Accessable Data Storage

In [32]:
import sqlite3 as db

In [33]:
URI = Path().resolve().parent
URI = URI.joinpath("db", "index.db")

In [35]:
def write_to_db(df, uri):
    conn = db.connect(uri)
    column_schema = [col.replace(" ", "_").lower() for col in df.columns]
    df.columns = column_schema
    df.to_sql(name='Sample', con=conn, if_exists='replace', index=False)
    conn.close()

In [36]:
write_to_db(data, URI)